In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# The pricing formula of Black-Scholes-Merton for European call/put options
"""
Inputs: 

S = spot price of the underlying stock

K = strike price

T = time to expiration

sigma = volatility of the option

r = risk-free interest rate

"""

def BS_call(S, K, T, sigma, r): 
    d1 = (np.log(S/K) + (r + ((sigma**2)/2)) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    C = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return C
    
def BS_put(S, K, T, sigma, r): 
    d1 = (np.log(S/K) + (r + ((sigma**2)/2)) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    P = -S * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(-d2)
    
    return P

In [3]:
# Example: IREN call option on 24/09/2025

S = 47.14
K = 47.00
T = 0.0833  # time to maturity 1m
iv = 1.1768 # implied volatility
r = 0.0425

print('IREN Oct24 47 Call: BS value', round(BS_call(S,K,T,iv,r),2), 'vs spot bid/ask 6.20/6.55')

IREN Oct24 47 Call: BS value 6.49 vs spot bid/ask 6.20/6.55


In [4]:
# Example: NBIS put option on 24/09/2025

S = 113.23
K = 120.00
T = 0.0833  # time to maturity 1m
iv = 0.8646 # implied volatility
r = 0.0425

print('NBIS Oct24 120 Put: BS value', round(BS_put(S,K,T,iv,r),2), 'vs spot bid/ask 14.90/15.50')

NBIS Oct24 120 Put: BS value 15.0 vs spot bid/ask 14.90/15.50


In [5]:
# Example: ORCL straddle on 24/09/2025

S = 308.46
K = 310.00
T = 0.0833    # time to maturity 1m
iv_c = 0.5678 # implied volatility of the call option
iv_p = 0.5490 # implied volatility of the put option
r = 0.0425

print('ORCL Oct24 310 straddle: BS value', round(BS_put(S,K,T,iv_c,r)+BS_put(S,K,T,iv_p,r),2), 'vs last price 39.15')

ORCL Oct24 310 straddle: BS value 40.1 vs last price 39.15


In [6]:
# Calculation of implied volatility using the Black-Scholes model

def IV(S, K, T, r, OP, option_type):
    """
    Inputs: 
    
    OP = Option price from the market
    option_type = 'call' or 'put'
    
    """
    e = 0.0001       # allowed error
    max_trial = 100  # max number of trials

    def BS(S, K, T, sigma, r, option_type):
        d1 = (np.log(S/K) + (r + ((sigma**2)/2)) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)

        if option_type == 'call':
            option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        else:
            option_price = -S * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(-d2)

        return option_price

    sigma_est = 0.5  # initial guess
    
    op_est = BS(S, K, T, sigma_est, r, option_type)
    diff = op_est - OP  # difference between the estimated price and the observed price
    counter = 0         # iteration counter

    while abs(diff) > e and counter < max_trial:
        d_1 = (np.log(S / K) + (r + 0.5 * sigma_est ** 2) * T) / (sigma_est * np.sqrt(T))
        vega = S * math.sqrt(T) * norm.pdf(d_1)
        op_est = BS(S, K, T, sigma_est, r, option_type)  
        diff = op_est - OP  

        if abs(diff) < e:
            break

        sigma_est = sigma_est - (diff / vega)  # correcting implied volatility
        counter += 1  

    return sigma_est